In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms
from torchvision import models

In [2]:
# 定义一些超参数
batch_size = 100  #批大小

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
#classes = ('洋紫荆','大滨菊','黄钟木', '木棉花', '向日葵')

In [5]:
#数据预处理
data_transform = transforms.Compose([
    transforms.Resize((224,224), 2),                           #对图像大小统一
    transforms.RandomHorizontalFlip(),                        #图像翻转
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[    #图像归一化
                             0.229, 0.224, 0.225])
         ])

test_dataset = torchvision.datasets.ImageFolder(root='drive/MyDrive/graduation project/work/devide data/test/', transform=data_transform)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=True, num_workers=0)
#花卉类别
#data_classes = classes

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [6]:
#选择CPU还是GPU的操作
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = models.AlexNet()
net.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 5),
        )
net.load_state_dict(torch.load('drive/MyDrive/graduation project/model training/flower.pt'))
#net.load_state_dict(torch.load('drive/MyDrive/app/alexnet_flower_500.pkl'))
#model.eval()


<All keys matched successfully>

In [7]:
correct = 0
total = 0
with torch.no_grad(): # 不进行计算图构建
    for data in test_loader:
        images, labels = data
        #images, labels = images.to(device), labels.to(device)
        images.to(device), labels.to(device)
        images, labels = Variable(images), Variable(labels)
        
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
 
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test images: 87 %
